# Problem

Is to identify products at risk of backorder before the event occurs so that business has time to react. 

## What is a Backorder?
Backorders are products that are temporarily out of stock, but a customer is permitted to place an order against future inventory. 
A backorder generally indicates that customer demand for a product or service exceeds a company’s capacity to supply it. Back orders are both good and bad. Strong demand can drive back orders, but so can suboptimal planning. 

## Data description

Data file contains the historical data for the 8 weeks prior to the week we are trying to predict. The data was taken as weekly snapshots at the start of each week. Columns are defined as follows:

    sku - Random ID for the product

    national_inv - Current inventory level for the part

    lead_time - Transit time for product (if available)

    in_transit_qty - Amount of product in transit from source

    forecast_3_month - Forecast sales for the next 3 months

    forecast_6_month - Forecast sales for the next 6 months

    forecast_9_month - Forecast sales for the next 9 months

    sales_1_month - Sales quantity for the prior 1 month time period

    sales_3_month - Sales quantity for the prior 3 month time period

    sales_6_month - Sales quantity for the prior 6 month time period

    sales_9_month - Sales quantity for the prior 9 month time period

    min_bank - Minimum recommend amount to stock

    potential_issue - Source issue for part identified

    pieces_past_due - Parts overdue from source

    perf_6_month_avg - Source performance for prior 6 month period

    perf_12_month_avg - Source performance for prior 12 month period

    local_bo_qty - Amount of stock orders overdue

    deck_risk - Part risk flag

    oe_constraint - Part risk flag

    ppap_risk - Part risk flag

    stop_auto_buy - Part risk flag

    rev_stop - Part risk flag

    went_on_backorder - Product actually went on backorder. This is the target value.
    
         Yes or 1 : Product backordered

         No or 0  : Product not backordered

# Loading the required libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, roc_curve, auc

import matplotlib.pyplot as plt

# Identify Right Error Metrics


## Function to calculate required metrics

In [2]:
def evaluate_model(act, pred):
    print("Confusion Matrix \n", confusion_matrix(act, pred))
    print("Accurcay : ", accuracy_score(act, pred))
    print("Recall   : ", recall_score(act, pred))
    print("Precision: ", precision_score(act, pred))    

# Loading the data

In [3]:
data = pd.read_csv("C:/Users/gsk44/OneDrive/Desktop/RandomForest/BackOrders.csv", header=0)

# Understand the Data - Exploratory Data Analysis (EDA)

## Number row and columns

In [4]:
data.shape

(61589, 23)

## First and last 5 rows

In [5]:
data.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1888279,117,NaN,0,0,0,0,0,0,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,1870557,7,2.0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,1475481,258,15.0,10,10,77,184,46,132,256,...,0,0.54,0.70,0,No,No,No,Yes,No,No
3,1758220,46,2.0,0,0,0,0,1,2,6,...,0,0.75,0.90,0,Yes,No,No,Yes,No,No
4,1360312,2,2.0,0,4,6,10,2,2,5,...,0,0.97,0.92,0,No,No,No,Yes,No,No


In [6]:
data.tail()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
61584,1397275,6,8.0,0,24,24,24,0,7,9,...,0,0.98,0.98,0,No,No,No,Yes,No,No
61585,3072139,130,2.0,0,40,80,140,18,108,230,...,0,0.51,0.28,0,No,No,No,Yes,No,No
61586,1909363,135,9.0,0,0,0,0,10,40,65,...,0,1.00,0.99,0,No,No,Yes,Yes,No,No
61587,1845783,63,NaN,0,0,0,0,452,1715,3425,...,0,-99.00,-99.00,1,No,No,No,No,No,Yes
61588,1200539,0,2.0,0,8,8,8,0,1,1,...,0,0.79,0.78,0,Yes,No,No,Yes,No,Yes


## Statistic summary

In [7]:
data.describe(include='all')

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
count,6.158900e+04,61589.000000,58186.000000,61589.000000,6.158900e+04,6.158900e+04,6.158900e+04,61589.000000,61589.000000,6.158900e+04,...,61589.000000,61589.000000,61589.000000,61589.000000,61589,61589,61589,61589,61589,61589
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2,2,2,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,No,No,Yes,No,No
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,48145,61577,53792,59303,61569,50296
mean,2.037188e+06,287.721882,7.559619,30.192843,1.692728e+02,3.150413e+02,4.535760e+02,44.742957,150.732631,2.835465e+02,...,1.605400,-6.264182,-5.863664,1.205361,NaN,NaN,NaN,NaN,NaN,NaN
std,6.564178e+05,4233.906931,6.498952,792.869253,5.286742e+03,9.774362e+03,1.420201e+04,1373.805831,5224.959649,8.872270e+03,...,42.309229,25.537906,24.844514,29.981155,NaN,NaN,NaN,NaN,NaN,NaN
min,1.068628e+06,-2999.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,-99.000000,-99.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.498574e+06,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.620000,0.640000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.898033e+06,10.000000,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000,4.000000e+00,...,0.000000,0.820000,0.800000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.314826e+06,57.000000,8.000000,0.000000,1.200000e+01,2.500000e+01,3.600000e+01,6.000000,17.000000,3.400000e+01,...,0.000000,0.960000,0.950000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


## Data type

In [8]:
data.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

# Data pre-processing

## Convert all the attributes to appropriate type

In [9]:
for col in ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 
            'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data[col] = data[col].astype('category')

### Re-display data type of each variable

In [10]:
data.dtypes

sku                  category
national_inv            int64
lead_time             float64
in_transit_qty          int64
forecast_3_month        int64
forecast_6_month        int64
forecast_9_month        int64
sales_1_month           int64
sales_3_month           int64
sales_6_month           int64
sales_9_month           int64
min_bank                int64
potential_issue      category
pieces_past_due         int64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty            int64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
went_on_backorder    category
dtype: object

### Statistic summary 
    

In [11]:
data.describe(include='all')

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
count,61589.0,61589.000000,58186.000000,61589.000000,6.158900e+04,6.158900e+04,6.158900e+04,61589.000000,61589.000000,6.158900e+04,...,61589.000000,61589.000000,61589.000000,61589.000000,61589,61589,61589,61589,61589,61589
unique,61589.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2,2,2,2,2
top,1068628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,No,No,Yes,No,No
freq,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,48145,61577,53792,59303,61569,50296
mean,NaN,287.721882,7.559619,30.192843,1.692728e+02,3.150413e+02,4.535760e+02,44.742957,150.732631,2.835465e+02,...,1.605400,-6.264182,-5.863664,1.205361,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,4233.906931,6.498952,792.869253,5.286742e+03,9.774362e+03,1.420201e+04,1373.805831,5224.959649,8.872270e+03,...,42.309229,25.537906,24.844514,29.981155,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,-2999.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,-99.000000,-99.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.620000,0.640000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,10.000000,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000,4.000000e+00,...,0.000000,0.820000,0.800000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,57.000000,8.000000,0.000000,1.200000e+01,2.500000e+01,3.600000e+01,6.000000,17.000000,3.400000e+01,...,0.000000,0.960000,0.950000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


## Delete sku attribute

In [12]:
np.size(np.unique(data.sku, return_counts=True)[0])

61589

In [13]:
data.drop('sku', axis=1, inplace=True)

## Missing Data


In [14]:
data.isnull().sum()

national_inv            0
lead_time            3403
in_transit_qty          0
forecast_3_month        0
forecast_6_month        0
forecast_9_month        0
sales_1_month           0
sales_3_month           0
sales_6_month           0
sales_9_month           0
min_bank                0
potential_issue         0
pieces_past_due         0
perf_6_month_avg        0
perf_12_month_avg       0
local_bo_qty            0
deck_risk               0
oe_constraint           0
ppap_risk               0
stop_auto_buy           0
rev_stop                0
went_on_backorder       0
dtype: int64

In [15]:
print (data.shape)

(61589, 22)


In [16]:
data = data.dropna(axis=0)

In [17]:
print(data.isnull().sum())
print(data.shape)

national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64
(58186, 22)


## Train and test split

### Target attribute distribution 

In [18]:
data['went_on_backorder'].value_counts()

No     47217
Yes    10969
Name: went_on_backorder, dtype: int64

In [19]:
data['went_on_backorder'].value_counts(normalize=True)*100

No     81.148386
Yes    18.851614
Name: went_on_backorder, dtype: float64

### Split the data into train and test


In [20]:
X = data.drop('went_on_backorder', axis=1)
y = data['went_on_backorder']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123) 

### Target attribute distribution after the split

In [21]:
print(pd.value_counts(y_train)/y_train.count() * 100)

print(pd.value_counts(y_test) /y_test.count() * 100)

No     81.185858
Yes    18.814142
Name: went_on_backorder, dtype: float64
No     81.060953
Yes    18.939047
Name: went_on_backorder, dtype: float64


## Convert categorical target attribute to numeric

In [22]:
le = LabelEncoder()

In [23]:
le.fit(y_train)

LabelEncoder()

In [24]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

### Target attribute distribution

In [25]:
y_train.size

40730

In [26]:
pd.value_counts(y_train)/y_train.size * 100

0    81.185858
1    18.814142
dtype: float64

## Checking the data types

In [27]:
X_train.dtypes

national_inv            int64
lead_time             float64
in_transit_qty          int64
forecast_3_month        int64
forecast_6_month        int64
forecast_9_month        int64
sales_1_month           int64
sales_3_month           int64
sales_6_month           int64
sales_9_month           int64
min_bank                int64
potential_issue      category
pieces_past_due         int64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty            int64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
dtype: object

## Standardize the numerical attributes
 

### Store numerical attributes name

In [28]:
num_attr = X_train.select_dtypes(include=['float64', 'int64']).columns
num_attr

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg',
       'local_bo_qty'],
      dtype='object')

### Using StandardScaler, standardize the numerical attributes 

In [29]:
scaler = StandardScaler()
scaler.fit(X_train[num_attr])

StandardScaler()

In [30]:
X_train_std = scaler.transform(X_train[num_attr])
X_test_std = scaler.transform(X_test[num_attr])

In [31]:
print(X_train_std.shape)
print(X_test_std.shape)

(40730, 15)
(17456, 15)


## Converting Categorical attributes to Numeric attributes

### Store categorical attributes name

In [32]:
cat_attr = X_train.select_dtypes(include=['category']).columns
cat_attr

Index(['potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk',
       'stop_auto_buy', 'rev_stop'],
      dtype='object')

### Using OneHotEncoder,  converting Categorical attributes to Numeric attributes 

In [33]:
enc = OneHotEncoder(drop = 'first')

enc.fit(X_train[cat_attr])

OneHotEncoder(drop='first')

In [34]:
X_train_ohe=enc.transform(X_train[cat_attr]).toarray()
X_test_ohe=enc.transform(X_test[cat_attr]).toarray()

## Concatenate attribute


In [35]:
X_train_con = np.concatenate([X_train_std, X_train_ohe], axis=1)
X_test_con = np.concatenate([X_test_std, X_test_ohe], axis=1)

In [36]:
X_train_con.shape

(40730, 21)

# Model building

## RandomForestClassifier Model

### Instantiate Model

In [37]:
clf1 = RandomForestClassifier()

### Train Model

In [38]:
clf1.fit(X_train_con, y_train)

RandomForestClassifier()

### List important features

In [39]:
importances = clf1.feature_importances_
print(importances)

indices = np.argsort(importances)[::-1]
print(indices)

[2.35431566e-01 3.20469254e-02 4.29879119e-02 1.01852452e-01
 9.35027945e-02 7.27084138e-02 4.37819348e-02 4.91584571e-02
 5.28802880e-02 5.97182401e-02 3.50964252e-02 1.07487468e-02
 6.30573186e-02 6.50374349e-02 2.07092484e-02 2.83737286e-04
 9.28603287e-03 6.50393101e-05 8.90085182e-03 2.73332098e-03
 1.28602305e-05]
[ 0  3  4  5 13 12  9  8  7  6  2 10  1 14 11 16 18 19 15 17 20]


In [40]:
ind_attr_names = np.concatenate((num_attr, cat_attr), axis=0)

pd.DataFrame([ind_attr_names[indices], np.sort(importances)[::-1]])

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,national_inv,forecast_3_month,forecast_6_month,forecast_9_month,perf_12_month_avg,perf_6_month_avg,sales_9_month,sales_6_month,sales_3_month,sales_1_month,...,min_bank,lead_time,local_bo_qty,pieces_past_due,deck_risk,ppap_risk,stop_auto_buy,potential_issue,oe_constraint,rev_stop
1,0.235432,0.101852,0.093503,0.072708,0.065037,0.063057,0.059718,0.05288,0.049158,0.043782,...,0.035096,0.032047,0.020709,0.010749,0.009286,0.008901,0.002733,0.000284,0.000065,0.000013


### Predict

In [41]:
train_pred = clf1.predict(X_train_con)
test_pred = clf1.predict(X_test_con)

### Evaluate

In [42]:
print("--Train--")
evaluate_model(y_train, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[33024    43]
 [   47  7616]]
Accurcay :  0.9977903265406335
Recall   :  0.9938666318674149
Precision:  0.9943856900378639
--Test--
Confusion Matrix 
 [[13601   549]
 [  605  2701]]
Accurcay :  0.9338909257561869
Recall   :  0.8169993950393224
Precision:  0.831076923076923


## Up-sampling 


### Instantiate SMOTE

In [43]:
smote = SMOTE(random_state=123)

### Fit Sample

In [44]:
X_train_sm, y_train_sm = smote.fit_resample(X_train_con, y_train)

In [45]:
np.unique(y_train, return_counts= True)

(array([0, 1]), array([33067,  7663], dtype=int64))

In [46]:
np.unique(y_train_sm, return_counts= True)

(array([0, 1]), array([33067, 33067], dtype=int64))

## RandomForestClassifier with up-sample data

### Instantiate Model

In [47]:
clf2 = RandomForestClassifier()

### Train the model

In [48]:
clf2.fit(X_train_sm, y_train_sm)

RandomForestClassifier()

### List important features

In [49]:
importances = clf2.feature_importances_
print(importances)

indices = np.argsort(importances)[::-1]
print(indices)

[2.07981872e-01 2.69264687e-02 3.25771853e-02 1.46571348e-01
 1.13720030e-01 1.19388438e-01 5.26720002e-02 4.99742085e-02
 4.16877517e-02 4.87904824e-02 2.94293695e-02 8.60224855e-03
 4.39591964e-02 4.46639521e-02 1.84126397e-02 1.87593429e-04
 6.29150529e-03 3.86903916e-05 5.89673922e-03 2.21671240e-03
 1.15683411e-05]
[ 0  3  5  4  6  7  9 13 12  8  2 10  1 14 11 16 18 19 15 17 20]


In [50]:
pd.DataFrame([ind_attr_names[indices], np.sort(importances)[::-1]])

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,national_inv,forecast_3_month,forecast_9_month,forecast_6_month,sales_1_month,sales_3_month,sales_9_month,perf_12_month_avg,perf_6_month_avg,sales_6_month,...,min_bank,lead_time,local_bo_qty,pieces_past_due,deck_risk,ppap_risk,stop_auto_buy,potential_issue,oe_constraint,rev_stop
1,0.207982,0.146571,0.119388,0.11372,0.052672,0.049974,0.04879,0.044664,0.043959,0.041688,...,0.029429,0.026926,0.018413,0.008602,0.006292,0.005897,0.002217,0.000188,0.000039,0.000012


### Predict

In [51]:
train_pred = clf2.predict(X_train_sm)
test_pred = clf2.predict(X_test_con)

### Evaluate

In [52]:
print("--Train--")
evaluate_model(y_train_sm, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[33011    56]
 [   51 33016]]
Accurcay :  0.9983820727613633
Recall   :  0.9984576768379351
Precision:  0.998306724721819
--Test--
Confusion Matrix 
 [[13310   840]
 [  407  2899]]
Accurcay :  0.9285632447296058
Recall   :  0.8768905021173624
Precision:  0.7753410002674512


## Hyper-parameter tuning using Grid Search and Cross Validation

### Parameters to test

In [53]:
param_grid = {"n_estimators" : [50, 100],
              "max_depth" : [1,5],
              "max_features" : [3, 5],
              "min_samples_leaf" : [1, 2, 4]}

### Instantiate Decision Tree

In [54]:
clf3 = RandomForestClassifier()

### Instantiate GridSearchCV 

In [55]:
clf_grid = GridSearchCV(clf2, param_grid, cv=2)

### Train DT using GridSearchCV

In [56]:
clf_grid.fit(X_train_sm, y_train_sm)

GridSearchCV(cv=2, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [1, 5], 'max_features': [3, 5],
                         'min_samples_leaf': [1, 2, 4],
                         'n_estimators': [50, 100]})

### Best Params

In [57]:
clf_grid.best_params_

{'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 2, 'n_estimators': 50}

### Predict 

In [58]:
train_pred = clf_grid.predict(X_train_sm)
test_pred = clf_grid.predict(X_test_con)

### Evaluate

In [59]:
print("--Train--")
evaluate_model(y_train_sm, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[27799  5268]
 [ 3158 29909]]
Accurcay :  0.8725920101611879
Recall   :  0.9044969304744912
Precision:  0.8502430565426273
--Test--
Confusion Matrix 
 [[11817  2333]
 [  484  2822]]
Accurcay :  0.8386228230980751
Recall   :  0.853599516031458
Precision:  0.5474296799224054


## Building RandomForestClassifier Model using Variable Importance

In [60]:
importances = clf_grid.best_estimator_.feature_importances_
indices = np.argsort(importances)[::-1]
print(indices)

select = indices[0:5]
print(select)

[ 0  4  3  5  7  6  2  8  9 14 10  1 12 13 11 18 16 19 20 15 17]
[0 4 3 5 7]


### Instantiate Model

In [61]:
clf4 = RandomForestClassifier(max_depth=5, max_features=5,
                              min_samples_leaf=2, n_estimators=100)

### Train the model

In [62]:
clf4.fit(X_train_sm[:,select], y_train_sm)

RandomForestClassifier(max_depth=5, max_features=5, min_samples_leaf=2)

### Predict

In [63]:
train_pred = clf4.predict(X_train_sm[:,select])
test_pred = clf4.predict(X_test_con[:,select])

### Evaluate

In [64]:
print("--Train--")
evaluate_model(y_train_sm, train_pred)
print("--Test--")
evaluate_model(y_test, test_pred)

--Train--
Confusion Matrix 
 [[28535  4532]
 [ 3675 29392]]
Accurcay :  0.8759034687150331
Recall   :  0.8888620074394411
Precision:  0.8664072632944229
--Test--
Confusion Matrix 
 [[12186  1964]
 [  531  2775]]
Accurcay :  0.8570692025664528
Recall   :  0.8393829401088929
Precision:  0.5855665752268411
